In [25]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.linear_model import LogisticRegression
import optuna
from sklearn.metrics import classification_report, accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
from imblearn.over_sampling import SMOTE
import glob
import re
import dataframe_image as dfi

In [19]:
# Define the function to read OMG data from a CSV file
def read_omg_csv(path_palm_data: str, 
                 n_omg_channels: int, 
                 n_acc_channels: int = 0, 
                 n_gyr_channels: int = 0, 
                 n_mag_channels: int = 0, 
                 n_enc_channels: int = 0,
                 button_ch: bool = True, 
                 sync_ch: bool = True, 
                 timestamp_ch: bool = True) -> pd.DataFrame:
    
    df_raw = pd.read_csv(path_palm_data, sep=' ', 
                         header=None, 
                         skipfooter=1, 
                         skiprows=1, 
                         engine='python')
    columns = np.arange(n_omg_channels).astype('str').tolist()
    
    for label, label_count in zip(['ACC', 'GYR', 'MAG', 'ENC'], 
                                  [n_acc_channels, n_gyr_channels, n_mag_channels, n_enc_channels]):
        columns = columns + ['{}{}'.format(label, i) for i in range(label_count)]
        
    if button_ch:
        columns = columns + ['BUTTON']
        
    if sync_ch:
        columns = columns + ['SYNC']
        
    if timestamp_ch:
        columns = columns + ['ts']
        
    df_raw.columns = columns
    
    return df_raw

In [20]:
def prepare_training_data(path_palm_data, path_protocol_data, path_meta_data, 
                          n_omg_channels=50, n_acc_channels=3, n_gyr_channels=3, 
                          n_mag_channels=0, n_enc_channels=6, 
                          standardize=True, normalize=True,
                          DO_REPLACE_TO_MOVING_AVERAGE=True, 
                          DO_CALCULATE_DERIVATIVE=True,
                          DO_SHIFT_GESTURE=True,
                          selected_channels='ALL'):
    """
    Подготовка данных для обучения и тестирования из файлов данных palm, protocol и meta.
    
    Аргументы:
    path_palm_data (str): Путь к файлу данных palm.
    path_protocol_data (str): Путь к файлу данных protocol.
    path_meta_data (str): Путь к файлу данных meta.
    n_omg_channels, n_acc_channels, и т.д. (int): Количество каналов сенсоров.
    standardize (bool): Если True, стандартизирует признаки.
    normalize (bool): Если True, нормализует признаки.
    DO_REPLACE_TO_MOVING_AVERAGE (bool): Если True, применяет скользящее среднее к данным OMG.
    DO_CALCULATE_DERIVATIVE (bool): Если True, вычисляет производные данных OMG.
    DO_SHIFT_GESTURE (bool): Если True, смещает целевой признак на максимальный скачок в данных.
    selected_channels (str): Выбор каналов данных ('OMG', 'ACC_GYR', 'ALL').
    
    Возвращает:
    tuple: Кортеж, содержащий данные для обучения и тестирования.
    """
    # Чтение данных OMG
    omg_data = read_omg_csv(path_palm_data, n_omg_channels, n_acc_channels, n_gyr_channels, 
                            n_mag_channels, n_enc_channels)
    
    # Чтение данных протокола и кодирование жестов
    gestures_protocol = pd.read_csv(path_protocol_data)
    le = LabelEncoder()
    gestures_protocol['gesture'] = le.fit_transform(
        gestures_protocol[[
            "Thumb", "Index", "Middle", "Ring", "Pinky",
            'Thumb_stretch', 'Index_stretch', 'Middle_stretch', 'Ring_stretch', 'Pinky_stretch'
        ]].apply(lambda row: str(tuple(row)), axis=1)
    )
    
    # Чтение метаинформации
    df_meta = pd.read_csv(path_meta_data)
    palm_file = path_palm_data.split('/')[-1]
    last_train_idx = df_meta[df_meta['montage'] == palm_file].to_dict(orient='records')[0]['last_train_idx']
    
    # Синхронизация меток жестов с данными OMG, используя канал SYNC
    y_cmd = np.array([gestures_protocol['gesture'].loc[s] for s in omg_data['SYNC'].values])
    
    # Подготовка названий признаков для данных OMG
    OMG_CH = [str(i) for i in range(n_omg_channels)]
    ACC_CH = ['ACC0', 'ACC1', 'ACC2']
    GYR_CH = ['GYR0', 'GYR1', 'GYR2']
    ALL_CH = OMG_CH + ACC_CH + GYR_CH

    # Выбор каналов в соответствии с параметром selected_channels
    if selected_channels == 'OMG':
        selected_features = OMG_CH
    elif selected_channels == 'ACC_GYR':
        selected_features = ACC_CH + GYR_CH
    else:
        selected_features = ALL_CH
    
    if DO_REPLACE_TO_MOVING_AVERAGE:
        # Замена на скользящее среднее
        for col in selected_features:
            omg_data[col] = omg_data[col].rolling(window=5).mean().bfill()
    
    if DO_CALCULATE_DERIVATIVE:
        # Вычисление производных данных
        OMG_DERIV = [f'{col}_deriv' for col in OMG_CH]
        for col in OMG_CH:
            omg_data[f'{col}_next'] = omg_data[col].shift(-1).ffill()
            omg_data[f'{col}_deriv'] = omg_data[f'{col}_next'] - omg_data[col]
        selected_features += OMG_DERIV

    if DO_SHIFT_GESTURE:
        # Смещение целевого признака
        id_max = 0
        cur_gesture = 0
        for i in range(y_cmd.shape[0]):
            if i < id_max:  # Пропускаем все значения до id_max
                continue
            prev_gesture = cur_gesture  # предыдущий жест
            cur_gesture = y_cmd[i]  # текущий жест
            if cur_gesture != prev_gesture:  # Если сменился жест
                id_max = omg_data[OMG_DERIV][i:i+35].abs().sum(axis=1).idxmax()  # Нахождение максимального скачка
                y_cmd[i:id_max] = prev_gesture  # Замена всех значений до id_max на предыдущий жест
    
    # Разделение данных на обучающие и тестовые наборы
    X_train = omg_data[selected_features].iloc[:last_train_idx].values
    y_train = y_cmd[:last_train_idx]
    X_test = omg_data[selected_features].iloc[last_train_idx:].values
    y_test = y_cmd[last_train_idx:]
    
    # Стандартизация и нормализация
    if standardize:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
    if normalize:
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    
    return (X_train, y_train), (X_test, y_test)

Далее будем фильтровать датасеты на основе метрики F1. Параметры для нейросети и SMOTE были подобраны ниже в ноутбуке при обучении вот на этих данных:
- '2023-05-31_17-14-41'
- '2023-05-05_17-57-30'
- '2023-10-25_08-52-30'
- '2023-10-18_11-16-21'
- '2023-09-29_09-20-47'


In [35]:
# функция для построения модели 
def build_model(X_train, y_train, best_params):
    """ Построение модели с использованием лучших параметров, найденных через Optuna. """
    smote = SMOTE(k_neighbors=best_params['k_neighbors'], sampling_strategy=best_params['sampling_strategy'], n_jobs=-1)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_resampled.shape[1],)),
        BatchNormalization(),
        Dropout(best_params['dropout_rate']),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(best_params['dropout_rate']),
        Dense(len(np.unique(y_resampled)), activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=best_params['learning_rate']),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_resampled, y_resampled, epochs=100, batch_size=best_params['batch_size'], validation_split=0.2, verbose=0)
    return model

# функция для рассчета метрики
def evaluate_dataset(model, X, y):
    """ Оценка датасета с помощью модели для получения F1-скора. """
    predictions = model.predict(X)
    predictions = np.argmax(predictions, axis=1)  # Получаем индексы максимальных значений для многоклассовой классификации
    f1 = f1_score(y, predictions, average='weighted')
    return f1

# функция фильтрации датасетов
def filter_datasets(base_path, quality_threshold, best_params):
    """
    Фильтрация датасетов на основе F1-скора, полученного от нейросети, с использованием строгого формата имен файлов.
    
    Args:
    base_path (str): Базовый путь к данным.
    model (Model): Предварительно обученная модель для оценки датасетов.
    quality_threshold (float): Порог F1-скора для включения датасета в обработку.
    
    Returns:
    dict: Список путей к файлам данных .palm, которые прошли фильтрацию.
    """
    # Компилируем паттерн для проверки имени файла
    pattern = re.compile(r'\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2}\.palm$')
    
    # Собираем список всех файлов, соответствующих шаблону
    all_files = [file for file in os.listdir(base_path) if pattern.match(file)]
    good_datasets = {}

    for file_name in all_files:
        print(f'---Processing {file_name}---')
        path_palm_data = f'{base_path}/{file_name}'
        path_protocol_data = f'{base_path}/{file_name}.protocol.csv'
        path_meta_data = f'{base_path}/meta_information.csv'
        # Загрузка и подготовка данных
        (X_train, y_train), (X_test, y_test) = prepare_training_data(path_palm_data, path_protocol_data, path_meta_data)
        
        #строим модель 
        model = build_model(X_train, y_train, best_params)
        
        # Обучение модели и оценка на валидационном наборе
        model.fit(X_train, y_train)
        f1 = evaluate_dataset(model, X_test, y_test)
        
        
        good_datasets[file_name] = f1

    return good_datasets

In [36]:
base_path = 'data'
best_params = {'k_neighbors': 10,
  'sampling_strategy': 'not majority',
  'learning_rate': 0.010594467077472143,
  'dropout_rate': 0.16185277449744218,
  'batch_size': 128}
quality_threshold = 0.7
good_datasets = filter_datasets(base_path, quality_threshold, best_params)
good_datasets = pd.DataFrame(list(good_datasets.items()), columns=['FileName', 'F1-Score'])

---Processing 2023-05-05_17-57-30.palm---
649/649 ━━━━━━━━━━━━━━━━━━━━ 0s 745us/step - accuracy: 0.9612 - loss: 0.1726
185/185 ━━━━━━━━━━━━━━━━━━━━ 0s 607us/step
---Processing 2023-05-07_15-19-05.palm---
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 719us/step - accuracy: 0.9494 - loss: 0.2356
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 593us/step
---Processing 2023-05-07_16-54-27.palm---
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 760us/step - accuracy: 0.9129 - loss: 0.4685
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step
---Processing 2023-05-12_19-17-00.palm---
423/423 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - accuracy: 0.8750 - loss: 0.4468
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 743us/step
---Processing 2023-05-15_16-16-08.palm---
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8795 - loss: 0.3896
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 804us/step
---Processing 2023-05-15_17-12-24.palm---
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 843us/step - accuracy: 0.7849 - loss: 0.6300
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step
---Processing 2023-05-19_12-04

In [37]:
good_datasets = good_datasets.sort_values(by='F1-Score')


In [38]:
dfi.export(good_datasets, 'good_datasets.png', chrome_path='C:\Program Files\Google\Chrome\Application\chrome.exe')

In [7]:
base_path = 'data'

# Словарь с файлами данных
data_files = {
    'file1': ('2023-05-31_17-14-41', 'palm'),
    'file2': ('2023-05-05_17-57-30', 'palm'),
    'file3': ('2023-10-25_08-52-30', 'palm'),
    'file4': ('2023-10-18_11-16-21', 'palm'),
    'file5': ('2023-09-29_09-20-47', 'palm')
}

def build_and_train_model(X_train, y_train, X_test, y_test, trial):
    # Настройка параметров SMOTE в Optuna
    k_neighbors = trial.suggest_int('k_neighbors', 2, 10)
    sampling_strategy = trial.suggest_categorical('sampling_strategy', ['auto', 'minority', 'not majority', 'all'])
    smote = SMOTE(k_neighbors=k_neighbors, sampling_strategy=sampling_strategy, n_jobs=-1)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

    # Настройка параметров модели с помощью Optuna
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.7)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    epochs = 100
    num_classes = len(np.unique(y_resampled))

    # Создание модели с динамически настроенными параметрами
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_resampled.shape[1],)),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_resampled, y_resampled, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    return test_accuracy

def build_final_model(X_train, y_train, best_params):
    # Применение SMOTE с лучшими параметрами, найденными Optuna
    smote = SMOTE(k_neighbors=best_params['k_neighbors'], sampling_strategy=best_params['sampling_strategy'], n_jobs=-1)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

    # Построение финальной модели с использованием лучших параметров
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_resampled.shape[1],)),
        BatchNormalization(),
        Dropout(best_params['dropout_rate']),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(best_params['dropout_rate']),
        Dense(len(np.unique(y_resampled)), activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=best_params['learning_rate']),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_resampled, y_resampled, epochs=100, batch_size=best_params['batch_size'], validation_split=0.2, verbose=0)
    return model

def run_model(use_optuna=True):
    all_X_train = []
    all_y_train = []
    all_X_test = []
    all_y_test = []
    

    for file_name, (date_folder, file_suffix) in data_files.items():
        # Формирование путей доступа к данным для каждого эксперимента
        path_palm_data = f'{base_path}/{date_folder}.{file_suffix}'
        path_protocol_data = f'{base_path}/{date_folder}.{file_suffix}.protocol.csv'
        path_meta_data = f'{base_path}/meta_information.csv'

        # Загрузка и подготовка данных
        (X_train, y_train), (X_test, y_test) = prepare_training_data(path_palm_data, path_protocol_data, path_meta_data)
        print(f'--- Running {file_name} ---')
        print(f'Shapes of data: {X_train.shape}, {y_train.shape}, {X_test.shape}, {y_test.shape}')

        all_X_train.append(X_train)
        all_y_train.append(y_train)
        all_X_test.append(X_test)
        all_y_test.append(y_test)

    # Объединение данных 
    X_train = np.concatenate(all_X_train, axis=0)
    y_train = np.concatenate(all_y_train, axis=0)
    X_test = np.concatenate(all_X_test, axis=0)
    y_test = np.concatenate(all_y_test, axis=0)
    
    print(f'--- Final shapes ---')
    print(f'Shapes of data: {X_train.shape}, {y_train.shape}, {X_test.shape}, {y_test.shape}')
    # Оптимизация параметров с помощью Optuna или использование предустановленных параметров
    if use_optuna:
        study = optuna.create_study(direction='maximize')
        objective = lambda trial: build_and_train_model(X_train, y_train, X_test, y_test, trial)
        study.optimize(objective, n_trials=50, n_jobs=-1)
        best_params = study.best_trial.params
    else:
        best_params = {
            'k_neighbors': 2, 
            'sampling_strategy': 'all', 
            'learning_rate': 0.00010322646860398609, 
            'dropout_rate': 0.2609382632562011, 
            'batch_size': 32
        }

    # Построение финальной модели и выполнение предсказаний
    final_model = build_final_model(X_train, y_train, best_params)
    y_pred = final_model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Сохранение и вывод результатов классификации
    report = classification_report(y_test, y_pred_classes)
    print('--- Combined Classification Report ---')
    print(report)
    print(f'--- Best Parameters for Combined Experiments ---')
    print(best_params)
    return best_params, final_model

run_model(use_optuna=False) # Установите `use_optuna=True`, чтобы использовать Optuna для оптимизации параметров

--- Running file1 ---
Shapes of data: (15679, 106), (15679,), (3889, 106), (3889,)
--- Running file2 ---
Shapes of data: (20756, 106), (20756,), (5892, 106), (5892,)
--- Running file3 ---
Shapes of data: (5674, 106), (5674,), (5494, 106), (5494,)
--- Running file4 ---
Shapes of data: (5677, 106), (5677,), (5497, 106), (5497,)


[I 2024-05-16 20:36:46,116] A new study created in memory with name: no-name-3c09267d-4e28-4185-a9c0-bb8523855c01


--- Running file5 ---
Shapes of data: (5690, 106), (5690,), (5505, 106), (5505,)
--- Final shapes ---
Shapes of data: (53476, 106), (53476,), (26277, 106), (26277,)


[I 2024-05-16 20:45:08,389] Trial 22 finished with value: 0.8146287798881531 and parameters: {'k_neighbors': 7, 'sampling_strategy': 'minority', 'learning_rate': 0.0872025848204322, 'dropout_rate': 0.17622305196662846, 'batch_size': 256}. Best is trial 22 with value: 0.8146287798881531.
[I 2024-05-16 20:45:09,121] Trial 24 finished with value: 0.8233816623687744 and parameters: {'k_neighbors': 9, 'sampling_strategy': 'minority', 'learning_rate': 1.766673634797885e-05, 'dropout_rate': 0.388913470139653, 'batch_size': 256}. Best is trial 24 with value: 0.8233816623687744.
[I 2024-05-16 20:53:40,808] Trial 32 finished with value: 0.8462914228439331 and parameters: {'k_neighbors': 7, 'sampling_strategy': 'minority', 'learning_rate': 0.0009871077961390274, 'dropout_rate': 0.6395507689164328, 'batch_size': 256}. Best is trial 32 with value: 0.8462914228439331.
[I 2024-05-16 20:56:43,008] Trial 14 finished with value: 0.8945465683937073 and parameters: {'k_neighbors': 5, 'sampling_strategy': 

822/822 ━━━━━━━━━━━━━━━━━━━━ 1s 633us/step
--- Combined Classification Report ---
              precision    recall  f1-score   support

           0       0.93      0.91      0.92     17294
           1       0.72      0.92      0.81      1712
           2       0.90      0.87      0.88      1885
           3       0.80      0.73      0.76      1937
           4       0.58      0.75      0.65      1779
           5       0.97      0.82      0.88      1670

    accuracy                           0.87     26277
   macro avg       0.82      0.83      0.82     26277
weighted avg       0.89      0.87      0.88     26277

--- Best Parameters for Combined Experiments ---
{'k_neighbors': 10, 'sampling_strategy': 'not majority', 'learning_rate': 0.010594467077472143, 'dropout_rate': 0.16185277449744218, 'batch_size': 128}


({'k_neighbors': 10,
  'sampling_strategy': 'not majority',
  'learning_rate': 0.010594467077472143,
  'dropout_rate': 0.16185277449744218,
  'batch_size': 128},
 <Sequential name=sequential_1, built=True>)

In [9]:
best_params = {'k_neighbors': 10,
  'sampling_strategy': 'not majority',
  'learning_rate': 0.010594467077472143,
  'dropout_rate': 0.16185277449744218,
  'batch_size': 128}

In [5]:
# if logistic_regression_optuna:
#     def objective(trial):
    
#         # Параметры для SMOTE
#         smote_k_neighbors = trial.suggest_int('smote_k_neighbors', 2, 15)
#         smote_sampling_strategy = trial.suggest_categorical('smote_sampling_strategy', ['auto', 'minority', 'not majority', 'all'])

#         # Параметры для логистической регрессии
#         C = trial.suggest_loguniform('C', 1e-5, 10)
#         max_iter = trial.suggest_int('max_iter', 1000, 10000)
#         penalty = trial.suggest_categorical('penalty', ['l1', 'l2', 'elasticnet'])
        
#         # Установка совместимого решателя в зависимости от выбранной регуляризации
#         if penalty == 'l1':
#             solver = 'liblinear' # liblinear поддерживает только l1 и l2
#         elif penalty == 'l2':
#             solver = trial.suggest_categorical('solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'])
#         elif penalty == 'elasticnet':
#             solver = 'saga'  # saga - единственный, который поддерживает elasticnet

#         # Применение SMOTE
#         smote = SMOTE(k_neighbors=smote_k_neighbors, sampling_strategy=smote_sampling_strategy)
#         X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

#         # Обучение модели логистической регрессии
#         model = LogisticRegression(C=C, max_iter=max_iter, penalty=penalty, solver=solver, l1_ratio=0.5 if penalty == 'elasticnet' else None)
#         model.fit(X_train_resampled, y_train_resampled)

#         # Оценка модели
#         score = f1_score(
#             y_test, 
#             model.predict(X_test), 
#             average = 'micro'
#         )
#         return score

#     # Создание исследования
#     study = optuna.create_study(direction='maximize')
#     study.optimize(objective, n_trials=100, n_jobs=-1)

#     print("Лучшие параметры:", study.best_trial.params)    

In [6]:
# if logistic_regression_optuna:
#     # Извлечение лучших параметров
#     best_params = study.best_trial.params
#     print("Лучшие параметры:", best_params)

#     # Применение SMOTE с лучшими параметрами
#     smote = SMOTE(k_neighbors=best_params['smote_k_neighbors'], sampling_strategy=best_params['smote_sampling_strategy'])
#     X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

#     # Выбор решателя в зависимости от типа регуляризации
#     if best_params['penalty'] == 'elasticnet':
#         solver = 'saga'  # saga - единственный решатель, поддерживающий elasticnet
#     elif best_params['penalty'] == 'l1':
#         solver = 'liblinear'  # liblinear - оптимальный выбор для l1 регуляризации
#     elif best_params['penalty'] == 'l2':
#         solver = best_params['solver'] 
#     else:  # 'none'
#         solver = 'lbfgs'  # lbfgs хорошо подходит для отсутствия регуляризации

#     # Построение и обучение модели логистической регрессии
#     model = LogisticRegression(
#         C=best_params['C'],
#         max_iter=best_params['max_iter'],
#         penalty=best_params['penalty'],
#         solver=solver,
#         l1_ratio=0.5 if best_params['penalty'] == 'elasticnet' else None,
#         multi_class='auto',
#         class_weight={0: 1, 1: 1, 2: 1, 3: 3, 4: 1, 5: 1}
#     )

#     model.fit(X_train_resampled, y_train_resampled)

#     #Делаем предсказание класса
#     y_pred = model.predict(X_test)

#     print(f'Метрики на валидационной выборке \n\
#     {classification_report(y_test, y_pred)}')